# Data Connection 核心组件


* Data Connection 是 LangChain 中处理外部数据的核心模块，包含以下主要组件：
* Document Loaders - 文档加载器
* Text Splitters - 文本分割器
* Embedding Models - 嵌入模型
* Vector Stores - 向量存储
* Retrievers - 检索器

In [1]:
"""
LangChain 0.3 Data Connection 完整示例
包含文档加载、文本分割、向量化、存储和检索的完整流程
"""

import os
from typing import List, Dict, Any
import asyncio

# 核心导入
from langchain_community.document_loaders import (
    TextLoader,
    PyPDFLoader,
    CSVLoader,
    JSONLoader,
    WebBaseLoader,
    DirectoryLoader
)
from langchain.text_splitter import (
    RecursiveCharacterTextSplitter,
    CharacterTextSplitter,
    TokenTextSplitter,
    MarkdownHeaderTextSplitter
)
from langchain_ollama import OllamaEmbeddings
from langchain_community.vectorstores import (
    FAISS,
    Chroma,
    Qdrant
)
from langchain_core.documents import Document
from langchain.retrievers import (
    BM25Retriever,
    EnsembleRetriever,
    MultiQueryRetriever
)
from langchain_ollama import ChatOllama

print("✅ 所有库导入成功")

USER_AGENT environment variable not set, consider setting it to identify your requests.


✅ 所有库导入成功


## 1. Document Loaders 示例

In [2]:

# 1. Document Loaders 示例
def document_loaders_example():
    """文档加载器示例"""
    print("\n" + "=" * 60)
    print("1. Document Loaders 文档加载器示例")
    print("=" * 60)

    # 1.1 文本文件加载
    print("\n1.1 文本文件加载")
    # 创建示例文本文件
    with open("sample.txt", "w", encoding="utf-8") as f:
        f.write("""
        人工智能（AI）是计算机科学的一个分支，致力于创建能够执行通常需要人类智能的任务的系统。
        机器学习是AI的一个子集，它使计算机能够从数据中学习而无需明确编程。
        深度学习是机器学习的一个子集，使用神经网络来模拟人脑的工作方式。
        """)

    loader = TextLoader("sample.txt", encoding="utf-8")
    documents = loader.load()
    print(f"加载的文档数量: {len(documents)}")
    print(f"文档内容预览: {documents[0].page_content[:100]}...")

    # 1.2 CSV文件加载
    print("\n1.2 CSV文件加载")
    import pandas as pd

    # 创建示例CSV
    df = pd.DataFrame({
        'name': ['张三', '李四', '王五'],
        'age': [25, 30, 35],
        'city': ['北京', '上海', '深圳'],
        'description': ['软件工程师', '数据科学家', '产品经理']
    })
    df.to_csv("sample.csv", index=False, encoding="utf-8")

    csv_loader = CSVLoader("sample.csv", encoding="utf-8")
    csv_docs = csv_loader.load()
    print(f"CSV文档数量: {len(csv_docs)}")
    print(f"CSV文档示例: {csv_docs[0].page_content}")

    # 1.3 JSON文件加载
    print("\n1.3 JSON文件加载")
    import json

    sample_data = [
        {"title": "Python编程", "content": "Python是一种高级编程语言", "category": "技术"},
        {"title": "数据分析", "content": "数据分析是从数据中提取洞察的过程", "category": "数据科学"}
    ]

    with open("sample.json", "w", encoding="utf-8") as f:
        json.dump(sample_data, f, ensure_ascii=False, indent=2)

    json_loader = JSONLoader("sample.json", jq_schema=".[].content")
    json_docs = json_loader.load()
    print(f"JSON文档数量: {len(json_docs)}")
    print(f"JSON文档示例: {json_docs[0].page_content}")

    # 1.4 目录批量加载
    print("\n1.4 目录批量加载")
    os.makedirs("docs", exist_ok=True)

    # 创建多个文档
    for i in range(3):
        with open(f"docs/doc_{i}.txt", "w", encoding="utf-8") as f:
            f.write(f"这是第{i + 1}个文档的内容。包含关于技术{i + 1}的详细信息。")

    dir_loader = DirectoryLoader("docs", glob="*.txt",
                                 loader_cls=TextLoader,
                                 loader_kwargs={"encoding": "utf-8"})
    dir_docs = dir_loader.load()
    print(f"目录文档数量: {len(dir_docs)}")

    return documents + csv_docs + json_docs + dir_docs

documents = document_loaders_example()


1. Document Loaders 文档加载器示例

1.1 文本文件加载
加载的文档数量: 1
文档内容预览: 
        人工智能（AI）是计算机科学的一个分支，致力于创建能够执行通常需要人类智能的任务的系统。
        机器学习是AI的一个子集，它使计算机能够从数据中学习而无需明确编程。
   ...

1.2 CSV文件加载
CSV文档数量: 3
CSV文档示例: name: 张三
age: 25
city: 北京
description: 软件工程师

1.3 JSON文件加载
JSON文档数量: 2
JSON文档示例: Python是一种高级编程语言

1.4 目录批量加载
目录文档数量: 3


## 2. Text Splitters 示例

In [3]:

# 2. Text Splitters 示例
def text_splitters_example(documents: List[Document]):
    """文本分割器示例"""
    print("\n" + "=" * 60)
    print("2. Text Splitters 文本分割器示例")
    print("=" * 60)

    # 创建长文本用于分割
    long_text = """
    人工智能的发展历程可以追溯到20世纪50年代。当时，计算机科学家开始探索让机器模拟人类智能的可能性。

    在1956年的达特茅斯会议上，人工智能这个术语首次被正式提出。这标志着AI作为一个独立学科的诞生。

    随后的几十年里，AI经历了多次起伏。60-70年代是第一个AI春天，专家系统得到了广泛应用。

    80年代末到90年代初，由于技术限制和过高期望，AI进入了所谓的"AI冬天"。

    21世纪以来，随着大数据、云计算和深度学习的发展，AI迎来了新的春天。

    今天，AI已经在图像识别、自然语言处理、推荐系统等领域取得了突破性进展。
    """

    long_doc = Document(page_content=long_text, metadata={"source": "ai_history"})

    # 2.1 递归字符分割器（推荐）
    print("\n2.1 RecursiveCharacterTextSplitter")
    recursive_splitter = RecursiveCharacterTextSplitter(
        chunk_size=200,
        chunk_overlap=50,
        length_function=len,
        separators=["\n\n", "\n", "。", "，", " ", ""]
    )

    recursive_chunks = recursive_splitter.split_documents([long_doc])
    print(f"递归分割块数: {len(recursive_chunks)}")
    for i, chunk in enumerate(recursive_chunks[:2]):
        print(f"块 {i + 1}: {chunk.page_content[:100]}...")

    # 2.2 字符分割器
    print("\n2.2 CharacterTextSplitter")
    char_splitter = CharacterTextSplitter(
        chunk_size=300,
        chunk_overlap=50,
        separator="\n\n"
    )

    char_chunks = char_splitter.split_documents([long_doc])
    print(f"字符分割块数: {len(char_chunks)}")

    # 2.3 Token分割器
    print("\n2.3 TokenTextSplitter")
    token_splitter = TokenTextSplitter(
        chunk_size=100,
        chunk_overlap=20
    )

    token_chunks = token_splitter.split_documents([long_doc])
    print(f"Token分割块数: {len(token_chunks)}")

    # 2.4 Markdown分割器
    print("\n2.4 MarkdownHeaderTextSplitter")
    markdown_text = """
# 人工智能概述

## 什么是人工智能
人工智能是计算机科学的一个分支。

## AI的应用领域

### 自然语言处理
NLP是AI的重要分支。

### 计算机视觉
计算机视觉让机器能够"看见"。

## 未来发展
AI将继续快速发展。
"""

    markdown_splitter = MarkdownHeaderTextSplitter(
        headers_to_split_on=[
            ("#", "Header 1"),
            ("##", "Header 2"),
            ("###", "Header 3"),
        ]
    )

    md_doc = Document(page_content=markdown_text)
    md_chunks = markdown_splitter.split_text(markdown_text)
    print(f"Markdown分割块数: {len(md_chunks)}")

    return recursive_chunks
# 2. 文本分割
chunks = text_splitters_example(documents)


2. Text Splitters 文本分割器示例

2.1 RecursiveCharacterTextSplitter
递归分割块数: 2
块 1: 人工智能的发展历程可以追溯到20世纪50年代。当时，计算机科学家开始探索让机器模拟人类智能的可能性。

    在1956年的达特茅斯会议上，人工智能这个术语首次被正式提出。这标志着AI作为一个独立学...
块 2: 80年代末到90年代初，由于技术限制和过高期望，AI进入了所谓的"AI冬天"。

    21世纪以来，随着大数据、云计算和深度学习的发展，AI迎来了新的春天。

    今天，AI已经在图像识别、自...

2.2 CharacterTextSplitter
字符分割块数: 1

2.3 TokenTextSplitter
Token分割块数: 7

2.4 MarkdownHeaderTextSplitter
Markdown分割块数: 4


## 3. Embedding Models 示例

In [4]:


# 3. Embedding Models 示例
def embedding_models_example():
    """嵌入模型示例"""
    print("\n" + "=" * 60)
    print("3. Embedding Models 嵌入模型示例")
    print("=" * 60)

    # 3.1 Ollama嵌入模型
    print("\n3.1 Ollama嵌入模型")
    try:
        embeddings = OllamaEmbeddings(
            base_url="http://localhost:11434",
            model="nomic-embed-text"  # 或使用其他嵌入模型
        )

        # 测试文本
        texts = [
            "人工智能是计算机科学的分支",
            "机器学习是AI的子集",
            "深度学习使用神经网络",
            "今天天气很好"
        ]

        # 生成嵌入向量
        text_embeddings = embeddings.embed_documents(texts)
        query_embedding = embeddings.embed_query("什么是人工智能？")

        print(f"文档嵌入数量: {len(text_embeddings)}")
        print(f"嵌入向量维度: {len(text_embeddings[0])}")
        print(f"查询嵌入维度: {len(query_embedding)}")

        # 计算相似度
        import numpy as np

        def cosine_similarity(a, b):
            return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

        print("\n相似度计算:")
        for i, text in enumerate(texts):
            similarity = cosine_similarity(query_embedding, text_embeddings[i])
            print(f"'{text}' 相似度: {similarity:.4f}")

        return embeddings

    except Exception as e:
        print(f"Ollama嵌入模型初始化失败: {e}")
        print("请确保Ollama服务正在运行并安装了嵌入模型")
        return None
# 3. 嵌入模型
embeddings = embedding_models_example()


3. Embedding Models 嵌入模型示例

3.1 Ollama嵌入模型
文档嵌入数量: 4
嵌入向量维度: 768
查询嵌入维度: 768

相似度计算:
'人工智能是计算机科学的分支' 相似度: 0.8551
'机器学习是AI的子集' 相似度: 0.6135
'深度学习使用神经网络' 相似度: 0.5818
'今天天气很好' 相似度: 0.5851


## 4. Vector Stores 示例

In [7]:


# 4. Vector Stores 示例
def vector_stores_example(chunks: List[Document], embeddings):
    """向量存储示例"""
    print("\n" + "=" * 60)
    print("4. Vector Stores 向量存储示例")
    print("=" * 60)

    if embeddings is None:
        print("跳过向量存储示例（嵌入模型不可用）")
        return None, None

    # 4.1 FAISS向量存储
    print("\n4.1 FAISS向量存储")
    try:
        # 创建FAISS向量存储
        faiss_vectorstore = FAISS.from_documents(chunks, embeddings)

        # 保存到本地
        faiss_vectorstore.save_local("faiss_index")
        print("✅ FAISS索引已保存")

        # 相似性搜索
        query = "人工智能的发展"
        similar_docs = faiss_vectorstore.similarity_search(query, k=3)

        print(f"\n查询: '{query}'")
        print("相似文档:")
        for i, doc in enumerate(similar_docs):
            print(f"{i + 1}. {doc.page_content[:100]}...")

        # 带分数的相似性搜索
        similar_docs_with_scores = faiss_vectorstore.similarity_search_with_score(query, k=3)
        print("\n带分数的搜索结果:")
        for i, (doc, score) in enumerate(similar_docs_with_scores):
            print(f"{i + 1}. 分数: {score:.4f} - {doc.page_content[:80]}...")

    except Exception as e:
        print(f"FAISS创建失败: {e}")
        faiss_vectorstore = None

    # 4.2 Chroma向量存储
    print("\n4.2 Chroma向量存储")
    try:
        chroma_vectorstore = Chroma.from_documents(
            chunks,
            embeddings,
            persist_directory="./chroma_db"
        )

        # 持久化
        chroma_vectorstore.persist()
        print("✅ Chroma数据库已持久化")

        # 搜索测试
        chroma_results = chroma_vectorstore.similarity_search("机器学习", k=2)
        print(f"Chroma搜索结果数量: {len(chroma_results)}")

    except Exception as e:
        print(f"Chroma创建失败: {e}")
        chroma_vectorstore = None

    return faiss_vectorstore, chroma_vectorstore

# 4. 向量存储
faiss_store, chroma_store = vector_stores_example(chunks, embeddings)


4. Vector Stores 向量存储示例

4.1 FAISS向量存储
✅ FAISS索引已保存

查询: '人工智能的发展'
相似文档:
1. 人工智能的发展历程可以追溯到20世纪50年代。当时，计算机科学家开始探索让机器模拟人类智能的可能性。

    在1956年的达特茅斯会议上，人工智能这个术语首次被正式提出。这标志着AI作为一个独立学...
2. 80年代末到90年代初，由于技术限制和过高期望，AI进入了所谓的"AI冬天"。

    21世纪以来，随着大数据、云计算和深度学习的发展，AI迎来了新的春天。

    今天，AI已经在图像识别、自...

带分数的搜索结果:
1. 分数: 0.4337 - 人工智能的发展历程可以追溯到20世纪50年代。当时，计算机科学家开始探索让机器模拟人类智能的可能性。

    在1956年的达特茅斯会议上，人工智能这个术语首...
2. 分数: 0.7495 - 80年代末到90年代初，由于技术限制和过高期望，AI进入了所谓的"AI冬天"。

    21世纪以来，随着大数据、云计算和深度学习的发展，AI迎来了新的春天。...

4.2 Chroma向量存储
✅ Chroma数据库已持久化
Chroma搜索结果数量: 2


C:\Users\34769\AppData\Local\Temp\ipykernel_20760\3854282456.py:51: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  chroma_vectorstore.persist()


## 5. Retrievers 示例

In [9]:


# 5. Retrievers 示例
def retrievers_example(vectorstore, chunks: List[Document]):
    """检索器示例"""
    print("\n" + "=" * 60)
    print("5. Retrievers 检索器示例")
    print("=" * 60)

    # 5.1 向量存储检索器
    print("\n5.1 向量存储检索器")
    if vectorstore:
        vector_retriever = vectorstore.as_retriever(
            search_type="similarity",
            search_kwargs={"k": 3}
        )

        results = vector_retriever.invoke("人工智能的应用")
        print(f"向量检索结果数量: {len(results)}")
        for i, doc in enumerate(results):
            print(f"{i + 1}. {doc.page_content[:100]}...")

    # 5.2 BM25检索器
    print("\n5.2 BM25检索器")
    try:
        bm25_retriever = BM25Retriever.from_documents(chunks)
        bm25_retriever.k = 3

        bm25_results = bm25_retriever.invoke("人工智能发展")
        print(f"BM25检索结果数量: {len(bm25_results)}")
        for i, doc in enumerate(bm25_results):
            print(f"{i + 1}. {doc.page_content[:100]}...")

    except Exception as e:
        print(f"BM25检索器创建失败: {e}")
        bm25_retriever = None

    # 5.3 集成检索器
    print("\n5.3 集成检索器")
    if vectorstore and bm25_retriever:
        try:
            ensemble_retriever = EnsembleRetriever(
                retrievers=[vector_retriever, bm25_retriever],
                weights=[0.7, 0.3]  # 向量搜索权重0.7，BM25权重0.3
            )

            ensemble_results = ensemble_retriever.invoke("机器学习技术")
            print(f"集成检索结果数量: {len(ensemble_results)}")

        except Exception as e:
            print(f"集成检索器创建失败: {e}")

    # 5.4 多查询检索器
    print("\n5.4 多查询检索器")
    if vectorstore:
        try:
            llm = ChatOllama(
                base_url="http://localhost:11434",
                model="gemma3:4b"
            )

            multi_query_retriever = MultiQueryRetriever.from_llm(
                retriever=vector_retriever,
                llm=llm
            )

            multi_results = multi_query_retriever.invoke("AI的未来发展趋势")
            print(f"多查询检索结果数量: {len(multi_results)}")

        except Exception as e:
            print(f"多查询检索器创建失败: {e}")
# 5. 检索器
retrievers_example(faiss_store, chunks)


5. Retrievers 检索器示例

5.1 向量存储检索器
向量检索结果数量: 2
1. 人工智能的发展历程可以追溯到20世纪50年代。当时，计算机科学家开始探索让机器模拟人类智能的可能性。

    在1956年的达特茅斯会议上，人工智能这个术语首次被正式提出。这标志着AI作为一个独立学...
2. 80年代末到90年代初，由于技术限制和过高期望，AI进入了所谓的"AI冬天"。

    21世纪以来，随着大数据、云计算和深度学习的发展，AI迎来了新的春天。

    今天，AI已经在图像识别、自...

5.2 BM25检索器
BM25检索结果数量: 2
1. 80年代末到90年代初，由于技术限制和过高期望，AI进入了所谓的"AI冬天"。

    21世纪以来，随着大数据、云计算和深度学习的发展，AI迎来了新的春天。

    今天，AI已经在图像识别、自...
2. 人工智能的发展历程可以追溯到20世纪50年代。当时，计算机科学家开始探索让机器模拟人类智能的可能性。

    在1956年的达特茅斯会议上，人工智能这个术语首次被正式提出。这标志着AI作为一个独立学...

5.3 集成检索器
集成检索结果数量: 2

5.4 多查询检索器
多查询检索结果数量: 2


## 6. 完整RAG流程示例

In [11]:

# 6. 完整RAG流程示例
def complete_rag_example():
    """完整的RAG流程示例"""
    print("\n" + "=" * 60)
    print("6. 完整RAG流程示例")
    print("=" * 60)

    try:
        # 6.1 准备数据
        documents = [
            Document(page_content="LangChain是一个用于构建LLM应用的框架", metadata={"source": "doc1"}),
            Document(page_content="向量数据库可以存储和检索高维向量", metadata={"source": "doc2"}),
            Document(page_content="RAG结合了检索和生成，提高了AI回答的准确性", metadata={"source": "doc3"}),
            Document(page_content="嵌入模型将文本转换为数值向量表示", metadata={"source": "doc4"})
        ]

        # 6.2 文本分割
        splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=20)
        chunks = splitter.split_documents(documents)

        # 6.3 创建嵌入和向量存储
        embeddings = OllamaEmbeddings(
            base_url="http://localhost:11434",
            model="nomic-embed-text"
        )

        vectorstore = FAISS.from_documents(chunks, embeddings)
        retriever = vectorstore.as_retriever(k=2)

        # 6.4 创建RAG链
        from langchain_core.prompts import ChatPromptTemplate
        from langchain_core.output_parsers import StrOutputParser
        from langchain_core.runnables import RunnablePassthrough

        llm = ChatOllama(
            base_url="http://localhost:11434",
            model="gemma3:4b"
        )

        prompt = ChatPromptTemplate.from_template("""
        基于以下上下文回答问题：

        上下文：{context}

        问题：{question}

        请提供准确、简洁的回答：
        """)

        def format_docs(docs):
            return "\n\n".join(doc.page_content for doc in docs)

        rag_chain = (
                {"context": retriever | format_docs, "question": RunnablePassthrough()}
                | prompt
                | llm
                | StrOutputParser()
        )

        # 6.5 测试RAG系统
        questions = [
            "什么是LangChain？",
            "向量数据库的作用是什么？",
            "RAG技术有什么优势？"
        ]

        for question in questions:
            print(f"\n问题: {question}")
            answer = rag_chain.invoke(question)
            print(f"回答: {answer}")

    except Exception as e:
        print(f"RAG流程执行失败: {e}")
complete_rag_example()


6. 完整RAG流程示例

问题: 什么是LangChain？
回答: LangChain是一个用于构建LLM应用的框架。


问题: 向量数据库的作用是什么？
回答: 向量数据库的作用是存储和检索高维向量。


问题: RAG技术有什么优势？
回答: RAG技术的优势在于它提高了AI回答的准确性。



## 7. 高级功能示例

In [12]:

# 7. 高级功能示例
def advanced_features_example():
    """高级功能示例"""
    print("\n" + "=" * 60)
    print("7. 高级功能示例")
    print("=" * 60)

    # 7.1 自定义文档加载器
    print("\n7.1 自定义文档加载器")

    class CustomLoader:
        def __init__(self, data_source):
            self.data_source = data_source

        def load(self):
            # 模拟从API或数据库加载数据
            documents = []
            for i, item in enumerate(self.data_source):
                doc = Document(
                    page_content=item["content"],
                    metadata={"id": i, "type": item["type"]}
                )
                documents.append(doc)
            return documents

    # 使用自定义加载器
    custom_data = [
        {"content": "Python是一种编程语言", "type": "技术"},
        {"content": "数据科学需要统计知识", "type": "科学"},
        {"content": "机器学习算法很重要", "type": "AI"}
    ]

    custom_loader = CustomLoader(custom_data)
    custom_docs = custom_loader.load()
    print(f"自定义加载器文档数量: {len(custom_docs)}")

    # 7.2 文档过滤和预处理
    print("\n7.2 文档过滤和预处理")

    def preprocess_documents(documents):
        """文档预处理函数"""
        processed_docs = []
        for doc in documents:
            # 清理文本
            content = doc.page_content.strip()
            content = content.replace("\n", " ").replace("\t", " ")

            # 过滤短文档
            if len(content) > 10:
                doc.page_content = content
                processed_docs.append(doc)

        return processed_docs

    processed_docs = preprocess_documents(custom_docs)
    print(f"预处理后文档数量: {len(processed_docs)}")
advanced_features_example()


7. 高级功能示例

7.1 自定义文档加载器
自定义加载器文档数量: 3

7.2 文档过滤和预处理
预处理后文档数量: 1


## 8. 性能优化示例

In [13]:
# 8. 性能优化示例
async def performance_optimization_example():
    """性能优化示例"""
    print("\n" + "=" * 60)
    print("8. 性能优化示例")
    print("=" * 60)

    # 8.1 批量处理
    print("\n8.1 批量嵌入处理")

    try:
        embeddings = OllamaEmbeddings(
            base_url="http://localhost:11434",
            model="nomic-embed-text"
        )

        # 大量文本
        texts = [f"这是第{i}个文档的内容" for i in range(10)]

        # 批量生成嵌入
        batch_embeddings = embeddings.embed_documents(texts)
        print(f"批量处理文档数量: {len(batch_embeddings)}")

        # 8.2 异步处理
        print("\n8.2 异步处理示例")

        async def async_embed_text(text):
            # 模拟异步嵌入
            await asyncio.sleep(0.1)
            return embeddings.embed_query(text)

        # 并发处理
        tasks = [async_embed_text(f"异步文本{i}") for i in range(5)]
        results = await asyncio.gather(*tasks)
        print(f"异步处理结果数量: {len(results)}")

    except Exception as e:
        print(f"性能优化示例失败: {e}")
performance_optimization_example()


<coroutine object performance_optimization_example at 0x000002743F9A9BE0>

## 主函数

In [14]:

# 主函数
def main():
    """运行所有示例"""
    print("🚀 LangChain 0.3 Data Connection 完整示例")
    print("=" * 80)

    # 1. 文档加载
    documents = document_loaders_example()

    # 2. 文本分割
    chunks = text_splitters_example(documents)

    # 3. 嵌入模型
    embeddings = embedding_models_example()

    # 4. 向量存储
    faiss_store, chroma_store = vector_stores_example(chunks, embeddings)

    # 5. 检索器
    retrievers_example(faiss_store, chunks)

    # 6. 完整RAG流程
    complete_rag_example()

    # 7. 高级功能
    advanced_features_example()

    print("\n🎉 所有示例运行完成！")

    # 清理临时文件
    cleanup_files()


def cleanup_files():
    """清理临时文件"""
    import shutil

    files_to_remove = ["sample.txt", "sample.csv", "sample.json"]
    dirs_to_remove = ["docs", "faiss_index", "chroma_db"]

    for file in files_to_remove:
        if os.path.exists(file):
            os.remove(file)

    for dir in dirs_to_remove:
        if os.path.exists(dir):
            shutil.rmtree(dir)

    print("🧹 临时文件已清理")


In [15]:

if __name__ == "__main__":
    # 运行同步示例
    main()

    # 运行异步示例
    # asyncio.run(performance_optimization_example())

🚀 LangChain 0.3 Data Connection 完整示例

1. Document Loaders 文档加载器示例

1.1 文本文件加载
加载的文档数量: 1
文档内容预览: 
        人工智能（AI）是计算机科学的一个分支，致力于创建能够执行通常需要人类智能的任务的系统。
        机器学习是AI的一个子集，它使计算机能够从数据中学习而无需明确编程。
   ...

1.2 CSV文件加载
CSV文档数量: 3
CSV文档示例: name: 张三
age: 25
city: 北京
description: 软件工程师

1.3 JSON文件加载
JSON文档数量: 2
JSON文档示例: Python是一种高级编程语言

1.4 目录批量加载
目录文档数量: 3

2. Text Splitters 文本分割器示例

2.1 RecursiveCharacterTextSplitter
递归分割块数: 2
块 1: 人工智能的发展历程可以追溯到20世纪50年代。当时，计算机科学家开始探索让机器模拟人类智能的可能性。

    在1956年的达特茅斯会议上，人工智能这个术语首次被正式提出。这标志着AI作为一个独立学...
块 2: 80年代末到90年代初，由于技术限制和过高期望，AI进入了所谓的"AI冬天"。

    21世纪以来，随着大数据、云计算和深度学习的发展，AI迎来了新的春天。

    今天，AI已经在图像识别、自...

2.2 CharacterTextSplitter
字符分割块数: 1

2.3 TokenTextSplitter
Token分割块数: 7

2.4 MarkdownHeaderTextSplitter
Markdown分割块数: 4

3. Embedding Models 嵌入模型示例

3.1 Ollama嵌入模型
文档嵌入数量: 4
嵌入向量维度: 768
查询嵌入维度: 768

相似度计算:
'人工智能是计算机科学的分支' 相似度: 0.8551
'机器学习是AI的子集' 相似度: 0.6135
'深度学习使用神经网络' 相似度: 0.5818
'今天天气很好' 相似度: 0.5851

4. Vector Stores 向量存储示例

4.1 FAISS向量存储
✅ FAISS索引已

PermissionError: [WinError 32] 另一个程序正在使用此文件，进程无法访问。: 'chroma_db\\44933678-68e8-40b1-b8d1-9afc146a6630\\data_level0.bin'